In [405]:
import numpy as np
from numpy.linalg import pinv
from numpy.linalg import det
import math
from math import pi
import time

In [406]:
#init variable
data = np.zeros([200,8])
train = np.zeros([140,8])
test = np.zeros([60,8])

In [407]:
def read(filepath):
    file = open(filepath)
    line = file.readline()
    n = 0
    while(line):
        line = file.readline()
        row = line.split()
        if(row!=[]):
            if(n<=139):
                for i in range(8):
                    train[n,i]=row[i]
                    data[n,i]=row[i]
            if(n>139):
                for i in range(8):
                    test[n-140,i]=row[i]
                    data[n,i]=row[i]
            n=n+1
    file.close()
    return data,train,test
#calculate mean
def mean(number):
    return sum(number)/len(number)
#calculate standard deviation for each feature of each class
def stdev(number):
    st = mean(number)
    sum = 0
    for n in range(len(number)):
        sum=sum + (number[n]-st)**2
    return math.sqrt(sum/len(number))

In [408]:
#seperate data to two different class for training
read("HW5_dataset.txt")
n0 = 0  #number of class 0
n1 = 0  #number of class 1
for label in train[:,0]:
    if label==0:
        n0 = n0+1
    if label==1:
        n1 = n1+1
s0 = np.zeros([n0,7])
s1 = np.zeros([n1,7])
s00 = 0
s11 = 0
for i in range(140):
    if(train[i,0]==0):
        s0[s00,:]=train[i,1:8]
        s00=s00+1
    if(train[i,0]==1):
        s1[s11,:]=train[i,1:8]
        s11=s11+1
#training data species0 s0, species1 s1

#possiblity of each class in sample
s0poss = s00/(s00+s11)
s1poss = s11/(s00+s11)
#calculate mu0 and mu1
mu0=np.zeros([7,1])
mu1=np.zeros([7,1])
for n in range(7):
    mu0[n,0]=mean(s0[:,n])
    mu1[n,0]=mean(s1[:,n])
#calculate sigma for class 0 and 1
#@parameter: sigma0, sigma1
sigma0 = np.zeros([7,7])
sigma1 = np.zeros([7,7])
for num in range(len(s0)):
    sigma0 = sigma0 + (s0[num:num+1,:].T-mu0)@(s0[num:num+1,:].T-mu0).T
for num in range(len(s1)):
    sigma1 = sigma1 + (s1[num:num+1,:].T-mu1)@(s1[num:num+1,:].T-mu1).T
sigma0=sigma0/len(s0)
sigma1=sigma1/len(s1)

In [409]:
#determine class 
#@structure  determine((7,1),T)
def determine(x):
    #x.reshape(7,1)
    left = 4*((pinv(sigma0+sigma1)@(mu1-mu0)).T@x)
    right = mu0.T@pinv(sigma0)@mu0-mu1.T@pinv(sigma1)@mu1
    if left-right<-9:
        return 0
    else:
        return 1

In [410]:
#accuracy of test data
correct = 0
for i in range(60):
    x = determine(test[i,1:8])
    if x == test[i,0]:
        correct = correct+1
    if x != test[i,0]:
        print(i+141,test[i])
print(correct/60)

1.0


In [411]:
#accuracy of training data
correct = 0
for i in range(140):
    x = determine(train[i,1:8])
    if x == train[i,0]:
        correct = correct + 1
    if x != train[i,0]:
        print(i,train[i,:])
print(correct/140)

1.0


In [412]:
#accuracy of entire data
correct = 0
wrong = 0
start=time.clock()
for i in range(200):
    x = determine(data[i,1:8])
    if x == data[i,0]:
        correct=correct+1
    if x!=data[i,0]:
        print(i,data[i,:])
print("run time: ",time.clock()-start,"s")
print("accuracy:",correct/200)

run time:  0.042671999999996046 s
accuracy: 1.0


In [413]:
#time of entire data
time_mean=[]
for i in range(200):
    start = time.clock()
    for i in range(200):
        determine(data[i,1:8])
    time_mean.append(time.clock()-start)
print(mean(time_mean))

0.03789636499999951


In [434]:
time_mean=[]
for n in range(200):
    start=time.clock()
    left = 4*((pinv(sigma0+sigma1)@(mu1-mu0)).T@data[:,1:8].T)
    right = mu0.T@pinv(sigma0)@mu0-mu1.T@pinv(sigma1)@mu1
    for i in range(200):
        if left[0,i]-right>-9:
            x = 1
        else:
            x = 0
    time_mean.append(time.clock()-start)
print(mean(time_mean))

0.0010512449999998807
